In [1]:
from transformers import BartConfig, BartModel

# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig()

# Initializing a model (with random weights) from the facebook/bart-large style configuration
model = BartModel(configuration)

# Accessing the model configuration
configuration = model.config

In [5]:
from transformers import AutoTokenizer, BartForConditionalGeneration
import recipe_parser as rp
import os
import pickle

if os.path.exists('pickle_files/recipe_descs.pkl'):
    descs = pickle.load(open('pickle_files/recipe_descs.pkl', 'rb'))
else:
    descs = rp.get_recipe_descs()

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

print(descs[1])


ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer(descs[2], max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Spread it on sandwiches, toss it with pasta, or treat yourself a single happy spoonful, but definitely absolutely positively make pesto any chance you get.


'Corn tortillas are made with just two ingredients: masa harina and water.'

In [3]:
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

TXT = "My friends are <mask> but they eat too many carbs."
input_ids = tokenizer([TXT], return_tensors="pt")["input_ids"]
logits = model(input_ids).logits

masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)

tokenizer.decode(predictions).split()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

['not', 'good', 'healthy', 'great', 'very']

### LLAMA

In [25]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-large')
set_seed(42)
Generated_Text = generator("The historic curious fact about pesto is that", max_length=200, num_return_sequences=1)
Generated_Text

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
Generated_Text[0]['generated_text']

"The curious fact about pesto is that it's more a tool than a food. If you buy the pesto from a store, then it's a tool. If you cook it – even an instant – and you see it go rancid, then you know the manufacturer's product sucked and it can go back to the store. (If you take the bait and eat a few pieces, you probably realize how bad it is in your stomach.) I have never bought it at a bakery. My boyfriend buys the pesto and I make him a couple of pizzas. He loves them.\n\nA lot of people probably don't know I make my pesto, but I'm the kind of person who makes everything by myself. I've never written a recipe. If I see a recipe in Cooks Illustrated, my favorite recipe website, and it says to make a pesto sauce instead, I make one by myself. (Sometimes I'll make the pesto and I'll make the"

In [24]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# generate speech by cloning a voice using default settings
tts.tts_to_file(text=Generated_Text[0]['generated_text'],
                file_path="audio_files/output.wav",
                speaker_wav="audio_files/Sample en.wav",
                language="en")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
["The curious fact about pesto is that it's more a tool than a food.", "If you buy the pesto from a store, then it's a tool.", "If you cook it – even an instant – and you see it go rancid, then you know the manufacturer's product sucked and it can go back to the store.", '(If you take the bait and eat a few pieces, you probably realize how bad it is in your stomach.)', 'I have never bought it at a bakery.', 'My boyfriend buys the pesto and I make him a couple of pizzas.', 'He loves them.', "A lot of people probably don't know I make my pesto, but I'm the kind of person who makes everything by myself.", "I've never written a recipe.", 'If I see a recipe in Cooks Illustrated, my favorite recipe website, and it says to make a pesto sauce instead, I make one by myself.', "(Sometimes I'll make the pesto and I'll make the"]
 > Processing time: 108.79297494888306
 > Real

'output.wav'